In [32]:
import numpy as np

In [33]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from stat_utils import run_trial  




def sample_true_params(process, ranges):
    """Sample one set of true params uniformly within the provided ranges."""
    return {k: rng.uniform(v[0], v[1]) for k, v in ranges[process].items()}

def simulate_data(process, params, n):
    """
    Simulate z (observations) as a (1, n) array.
    For 'biased_*', the bias is added directly to the observations.
    """
    if process in ('centered_iid', 'biased_iid'):
        sigma_obs = params['sigma_observation']
        bias = params.get('bias', 0.0)
        z = rng.normal(loc=bias, scale=sigma_obs, size=n)
        return z.reshape(1, n)

    # Kalman-type processes: x_t = a x_{t-1} + w_t, z_t = x_t + v_t (+ bias)
    a = params['a']
    sigma_p = params['sigma_process']
    sigma_o = params['sigma_observation']
    bias = params.get('bias', 0.0)

    x = np.zeros(n)
    for t in range(1, n):
        x[t] = a * x[t-1] + rng.normal(0.0, sigma_p)
    z = x + rng.normal(0.0, sigma_o, size=n) + bias
    return z.reshape(1, n)

def build_filt_params_for(process, theta_dict):
    """
    Given parameters (as a dict), return the filter parameter dict
    {F, H, Q, R} with shapes consistent with run_trial().
    All models are 1D state and 1D observation.
    """
    H = np.array([[1.0]])
    if process in ('centered_iid', 'biased_iid'):
        F = np.array([[1.0]])
        Q = np.array([[0.0]])
        R = np.array([[theta_dict['sigma_observation'] ** 2]])
    else:
        F = np.array([[theta_dict['a']]])
        Q = np.array([[theta_dict['sigma_process'] ** 2]])
        R = np.array([[theta_dict['sigma_observation'] ** 2]])
    return dict(F=F, H=H, Q=Q, R=R)

def total_loglik(process, data_1xn, theta_vec):
    """
    Compute total log-likelihood for given process and parameters.
    For biased models, subtract the bias from the data before filtering.
    """
    names = param_order[process]
    theta = {n: float(theta_vec[i]) for i, n in enumerate(names)}

    # Preprocess data for bias terms: z' = z - bias
    if 'bias' in theta:
        sim_data = (data_1xn - theta['bias'])
    else:
        sim_data = data_1xn

    if 'a' in theta: #TODO: find a nicer way to check for kalman vs iid
        # Kalman filter parameters
        filt_params = build_filt_params_for(process, theta)

        # Run the provided validated likelihood (no missing data here)
        res = run_trial(
            filt_params=filt_params,
            sim_data=sim_data,
            Tmax=sim_data.shape[1],
            missing_prob=0.0,
            discard_1st_step_stats=False
        )   
    else:
        # IID case: closed-form log-likelihood
        sigma_o = theta['sigma_observation']
        residuals = sim_data.flatten()
        n = residuals.size
        ll = -0.5 * n * np.log(2 * np.pi * sigma_o**2) - 0.5 * np.sum(residuals**2) / (sigma_o**2)
        res = {'total_ll': ll}

    ll = res['total_ll']
    # Be robust to numerical issues
    if ll is None or not np.isfinite(ll):
        return -np.inf
    return ll

def estimate_params(process, data_1xn, true_params):
    """Local MLE using L-BFGS-B with bounds, initialized at the true parameters."""
    names = param_order[process]
    x0 = np.array([true_params[n] for n in names], dtype=float)
    bounds = [param_ranges[process][n] for n in names]

    def objective(theta):
        ll = total_loglik(process, data_1xn, theta)
        # minimize negative log-likelihood
        return -ll

    result = minimize(
        objective,
        x0=x0,
        method='Nelder-Mead', #'L-BFGS-B',
        # bounds=bounds
    )

    est = {n: float(result.x[i]) for i, n in enumerate(names)}
    ll_at_est = -float(result.fun)
    ll_at_true = total_loglik(process, data_1xn, x0)

    out = {
        'success': bool(result.success),
        'status': int(result.status),
        'message': str(result.message),
        'nit': int(result.nit),
        'nfev': int(result.nfev),
        'll_at_true': float(ll_at_true),
        'll_at_mle': float(ll_at_est),
    }
    # attach parameter maps
    out.update({f'true_{k}': float(true_params[k]) for k in names})
    out.update({f'est_{k}': float(est[k]) for k in names})
    return out

# # ============================
# # Main loop: simulate & fit
# # ============================
# results = []

# for proc in processes:
#     for rep in range(n_repeats):
#         # 1) sample true params
#         theta_true = sample_true_params(proc, param_ranges)

#         # 2) simulate data
#         z = simulate_data(proc, theta_true, n_samples)  # shape (1, n_samples)

#         # 3) fit starting from truth (local optimization)
#         fit = estimate_params(proc, z, theta_true)
#         fit.update({'process': proc, 'repeat': rep})

#         results.append(fit)

# # Convert to a DataFrame for convenience
# df_results = pd.DataFrame(results)

# # Example: show a quick summary per process
# summary_cols = ['process', 'success', 'll_at_true', 'll_at_mle'] + \
#                [c for c in df_results.columns if c.startswith('true_')] + \
#                [c for c in df_results.columns if c.startswith('est_')]
# print(df_results[summary_cols].groupby('process').agg(['mean']))
# # You can also save: df_results.to_csv('kalman_mle_results.csv', index=False)


In [34]:
# --- BIC helpers -------------------------------------------------------------

def bic_from_ll(total_loglik, k, n):
    """Schwarz BIC = -2 * ln L + k * ln n."""
    if total_loglik is None or not np.isfinite(total_loglik):
        return np.inf
    return -2.0 * float(total_loglik) + k * np.log(n)

def midpoint(lo, hi):
    return 0.5 * (lo + hi)

def make_init_params(target_process, true_params):
    """
    Build an initializer for 'target_process' using any overlapping true params.
    Non-overlapping params are set to the midpoint of their allowed range.
    """
    init = {}
    for p in param_order[target_process]:
        if p in true_params:
            init[p] = float(true_params[p])
        else:
            lo, hi = param_ranges[target_process][p]
            init[p] = midpoint(lo, hi)
    return init



In [35]:

rng = np.random.default_rng(1234)

# Parameter order per process (used to pack/unpack vectors)
param_order = {
    'centered_iid':      ['sigma_observation'],
    'biased_iid':        ['sigma_observation', 'bias'],
    'centered_kalman':   ['sigma_process', 'a', 'sigma_observation'],
    'biased_kalman':     ['sigma_process', 'a', 'sigma_observation', 'bias'],
}

# processes = ['centered_iid', 'biased_iid', 'centered_kalman', 'biased_kalman'] # dic of tuples (min, max) for each parameter
# param_ranges = {
#     'centered_iid': {'sigma_observation': (0.1, 1.0)},
#     'biased_iid': {'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)},
#     'centered_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 0.95), 'sigma_observation': (0.1, 1.0)},  # kalman process
#     'biased_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 2.0), 'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)}
# }

for var_frac_ in [None]: #[0.2, 0.5, 0.8]:
    if var_frac_ is None:
        processes = ['centered_iid', 'biased_iid', 'centered_kalman', 'biased_kalman'] # dic of tuples (min, max) for each parameter
        param_ranges = {
        'centered_iid': {'sigma_observation': (0.1, 1.0)},
        'biased_iid': {'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)},
        'centered_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 0.95), 'sigma_observation': (0.1, 1.0)},  # kalman process
        'biased_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 2.0), 'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)}}
    else:
        processes = ['centered_iid', 'centered_kalman'] # dic of tuples (min, max) for each parameter
        print(f"\n=== Results for observation variance fraction: {var_frac_} ===")
        ee=1e-6
        a_ = 0.8
        sigma_o_ = var_frac_**0.5
        sigma_p_ = np.sqrt((1-sigma_o_**2)*(1 - a_**2))  # to have stationary variance 0.5
        param_ranges = {
            'centered_iid': {'sigma_observation': (1.0, 1.0+ee)},
            # 'biased_iid': {'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)},
            'centered_kalman': {'sigma_process': (sigma_p_, sigma_p_+ee), 'a': (a_, a_+ee), 'sigma_observation': (sigma_o_, sigma_o_+ee)},  # kalman process
            # 'biased_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 2.0), 'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)}
        }

    n_samples = 100
    n_repeats = 20

    # --- BIC evaluation and confusion matrix ------------------------------------

    bic_records = []
    variance_records = {}
    skip_fitting = False  # set to True to only compute variances of simulated data
    for true_proc in processes:
        variance_records[true_proc] = []
        for rep in range(n_repeats):
            # 1) draw true params for the generator
            theta_true = sample_true_params(true_proc, param_ranges)

            # 2) simulate one dataset from the true process
            z = simulate_data(true_proc, theta_true, n_samples)  # shape (1, n_samples)
            variance_records[true_proc].append(np.mean(z**2))
            # 3) fit each candidate model (local optimization, init ~ true where possible)
            if not skip_fitting:
                for cand in processes:
                    init_params = make_init_params(cand, theta_true)
                    fit = estimate_params(cand, z, init_params)

                    k = len(param_order[cand])
                    ll = fit['ll_at_mle']

                    # if 'est_a' in fit:
                    #     discount_fac = 1-fit['est_a']
                    # else:
                    #     a_hat = n_samples-1
                        
                    bic = bic_from_ll(ll, k, n_samples)
                    aic = 2*k - 2*ll

                    bic_records.append({
                        'true_process': true_proc,
                        'repeat': rep,
                        'candidate': cand,
                        'k': k,
                        'll_at_mle': ll,
                        'nll_at_mle': -ll,
                        'bic': bic,
                        'aic': aic,
                        'success': fit['success']
                    })
    if not skip_fitting:
        # Results per fit
        df_bic = pd.DataFrame(bic_records)

        for criterion in ['bic', 'aic' , 'nll_at_mle']:
            print(f"\n--- Model selection using criterion: {criterion} ---")
            # Pick the BIC winner for each (true_process, repeat)
            idx_best = df_bic.groupby(['true_process', 'repeat'])[criterion].idxmin()
            df_choice = df_bic.loc[idx_best, ['true_process', 'repeat', 'candidate']] \
                            .rename(columns={'candidate': 'selected_model'})

            # Confusion table (rows: true process, columns: BIC-selected)
            confusion = pd.crosstab(df_choice['true_process'], df_choice['selected_model'])

            # (optional) overall accuracy
            overall_acc = np.trace(confusion.values) / confusion.values.sum()

            print(f"{criterion} confusion table:")
            print(confusion)
            print(f"\nOverall --{criterion}-- selection accuracy: {overall_acc:.3f}")

            # # (optional) per-true-process accuracy
            # per_true_acc = (confusion.values.diagonal() / confusion.sum(axis=1).values)
            # per_true = pd.Series(per_true_acc, index=confusion.index, name='accuracy')
            # # print("\nPer-true-process accuracy:")
            # # print(per_true.to_string())



--- Model selection using criterion: bic ---
bic confusion table:
selected_model   biased_iid  biased_kalman  centered_iid  centered_kalman
true_process                                                             
biased_iid               20              0             0                0
biased_kalman             3              8             0                9
centered_iid              0              0            20                0
centered_kalman           3              0             4               13

Overall --bic-- selection accuracy: 0.762

--- Model selection using criterion: aic ---
aic confusion table:
selected_model   biased_iid  biased_kalman  centered_iid  centered_kalman
true_process                                                             
biased_iid               18              2             0                0
biased_kalman             2             12             0                6
centered_iid              4              0            15                1
centered_

In [ ]:
'''
an output for the setting with very restricted params:
        a_ = 0.8
        sigma_o_ = var_frac_**0.5
        sigma_p_ = np.sqrt((1-sigma_o_**2)*(1 - a_**2))  # to have stationary variance 0.5
        param_ranges = {
            'centered_iid': {'sigma_observation': (1.0, 1.0+ee)},
            # 'biased_iid': {'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)},
            'centered_kalman': {'sigma_process': (sigma_p_, sigma_p_+ee), 'a': (a_, a_+ee), 'sigma_observation': (sigma_o_, sigma_o_+ee)},  # kalman process
            # 'biased_kalman': {'sigma_process': (0.1, 1.0), 'a': (0.5, 2.0), 'sigma_observation': (0.1, 1.0), 'bias': (0.1, 1.0)}
        }

'''

In [40]:
# === Results for observation variance fraction: 0.2 ===

# --- Model selection using criterion: bic ---
# bic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               20                0
# centered_kalman             0               20

# Overall --bic-- selection accuracy: 1.000

# --- Model selection using criterion: aic ---
# aic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               19                1
# centered_kalman             0               20

# Overall --aic-- selection accuracy: 0.975

# --- Model selection using criterion: nll_at_mle ---
# nll_at_mle confusion table:
# selected_model   centered_kalman
# true_process                    
# centered_iid                  20
# centered_kalman               20

# Overall --nll_at_mle-- selection accuracy: 0.500

# === Results for observation variance fraction: 0.5 ===

# --- Model selection using criterion: bic ---
# bic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               20                0
# centered_kalman             4               16

# Overall --bic-- selection accuracy: 0.900

# --- Model selection using criterion: aic ---
# aic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               19                1
# centered_kalman             1               19

# Overall --aic-- selection accuracy: 0.950

# --- Model selection using criterion: nll_at_mle ---
# nll_at_mle confusion table:
# selected_model   centered_kalman
# true_process                    
# centered_iid                  20
# centered_kalman               20

# Overall --nll_at_mle-- selection accuracy: 0.500

# === Results for observation variance fraction: 0.8 ===

# --- Model selection using criterion: bic ---
# bic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               20                0
# centered_kalman            17                3

# Overall --bic-- selection accuracy: 0.575

# --- Model selection using criterion: aic ---
# aic confusion table:
# selected_model   centered_iid  centered_kalman
# true_process                                  
# centered_iid               19                1
# centered_kalman            11                9

# Overall --aic-- selection accuracy: 0.700

# --- Model selection using criterion: nll_at_mle ---
# nll_at_mle confusion table:
# selected_model   centered_kalman
# true_process                    
# centered_iid                  20
# centered_kalman               20

In [ ]:

# --- Model selection using criterion: bic ---
# bic confusion table:
# selected_model   biased_iid  biased_kalman  centered_iid  centered_kalman
# true_process                                                             
# biased_iid               20              0             0                0
# biased_kalman             3              8             0                9
# centered_iid              0              0            20                0
# centered_kalman           3              0             4               13

# Overall --bic-- selection accuracy: 0.762

# --- Model selection using criterion: aic ---
# aic confusion table:
# selected_model   biased_iid  biased_kalman  centered_iid  centered_kalman
# true_process                                                             
# biased_iid               18              2             0                0
# biased_kalman             2             12             0                6
# centered_iid              4              0            15                1
# centered_kalman           2              3             0               15

# Overall --aic-- selection accuracy: 0.750

# --- Model selection using criterion: nll_at_mle ---
# nll_at_mle confusion table:
# selected_model   biased_iid  biased_kalman  centered_kalman
# true_process                                               
# biased_iid                4             16                0
# biased_kalman             0             19                1
# centered_iid              4             15                1
# centered_kalman           0             20                0

# Overall --nll_at_mle-- selection accuracy: 0.300